In [ ]:
!pip install tweepy==3.9.0 pymongo

In [ ]:
from __future__ import print_function
import tweepy
import pandas as pd
import json
import time
from pymongo import MongoClient
from urllib3.exceptions import ProtocolError

MONGO_HOST= '' #connexion string to mongdb

try:
    client = MongoClient(MONGO_HOST)
    db = client.db # Use database (If it doesn't exist, it will be created)
except Exception as e:
    print(e)

In [ ]:
KEYWORDS = ["insulin","#insulin","diabetes","#diabetes","diabetic","#diabetic","#diabeticproblems","blood glucose","#bloodglucose","#diabeticstruggles",    
             "#lifewithdiabetes","#type1diabetes","#type2diabetes","#insulin4all","#thisisdiabetes","#stopdiabetes","#fingerprick","diabetes AND pain",
            "insuline","#insuline","Diabète","#Diabète","diabétique","#diabétique","#problèmedediabétique","glucose sanguin","#glucosesanguin",
             "#maviedediabétique","#diabètetype1","#diabète de type 2","#diabètedetype1","#diabetetype2","#mondiabete","glucose",
             "Diabetiker","#Diabetiker","#diabetikerproblem","Blutzucker","#blutzucker","#meindiabetes","#Diabetikerleben","#diabetestyp1",
            "#Typ1Diabetes","#diabetestyp2","#Typ2Diabetes",
             "insulina","#insulina","diabético","#diabético","#diabeticproblems","glucosa en la sangre","#glucosaenlasangre","#Diabetestipo1",
             "#Diabetestipo2","#detenerladiabetes","#estoesdiabetes",
     "インシュリン","#インシュリン","糖尿病","#糖尿病","糖尿病の","#糖尿病の","糖尿病の問題","血糖","#血糖","1型糖尿病","#1型糖尿病","2型糖尿病",
     "＃2型糖尿病","#私の糖尿病",
     "กลูโคส","#กลูโคส","อินซูลิน","#อินซูลิน","โรคเบาหวาน","#โรคเบาหวาน","การต่อสู้กับโรคเบาหวาน","ระดับน้ำตาลในเลือด","#ระดับน้ำตาลในเลือด",
     "การต่อสู้กับโรคเบาหวาน","โรคเบาหวานประเภท 1","โรคเบาหวานประเภท 2","โรคเบาหวานของฉัน",
     "gula darah","#guladarah","perjuangan diabetes","hidup dengan diabetes","diabetes tipe 1","#diabetestipe1","diabetes tipe 2","#diabetestipe2",
     "인슐린","#인슐린","당뇨병","#당뇨병","당뇨병 환자","#당뇨병 환자","혈당","#혈당","당뇨병 투쟁","제 1 형 당뇨병","제 2 형 당뇨병",
     "dyabetis","#dyabetis","dyabetiko","#dyabetiko","asukal sa dugo","#asukalsadugo","problema sa diabetes",
     "#problemasdiabéticos","glicose no sangue","#glicosenosangue","#vidacomdiabetes",
     "इंसुलिन","#इंसुलिन","मधुमेह","#मधुमेह","मधुमेह की समस्या","रक्त ग्लूकोज","मधुमेह के संघर्ष","के साथ जीवन मधुमेह","टाइप 1 मधुमेह","टाइप 2 मधुमेह","#madhumeh",
     "ensülin","#ensülin","diyabet","#diyabet","şeker hastası","#şekerhastası","kan şekeri","#kan şekeri","tip 1 diyabet","#tip1diyabet",
     "tip 2 diyabet","#tip2diyabet",
     "لأنسولين","#الأنسولين","داء السكري","#داءالسكري","مريض بالسكر","#مريض بالسكر","جلوكوز الدم","يكافح السكري","مرض السكر النوع 1","مرض السكر النوع 2", 
    "инсулин","#инсулин","диабет","#диабет","диабетический","#диабетический","содержание глюкозы в крови","#жизньсдиабетом","диабет 1 типа",
     "#диабет1типа","диабет 2 типа","#диабет2типа",
     "glucosio nel sangue","#glucosionelsangue","diabete di tipo 1","#diabeteditipo1","diabete di tipo 2","#diabeteditipo2",
     "胰岛素","#胰岛素","糖尿病","#胰岛素","糖尿病患者","#糖尿病患者","血糖","#血糖","#2型糖尿病","#1型糖尿病",
     "pesakit kencing manis","#pesakitkencingmanis","glukosa darah","#glukosadarah","diabetes jenis 1","diabetes jenis 2",
     "cukrzyca","#cukrzyca","cukrzycowy","#cukrzycowy","glukoza we krwi","#glukozawekrwi","#cukrzycatypu1","#cukrzycatypu2","#mojacukrzyca",
     "życie z cukrzycą",
     "suikerziekte","#suikerziekte","bloed glucose","#bloedglucose","#diabetestype1","#diabetestype2",
     "diabetis","#diabetis","glucosa a la sang","diabetis tipus 1","#diabetistipus1","diabetis tipus 2","#diabetistipus2",
     "blodsukker","#blodsukker","#mindiabetes",
     "ινσουλίνη","#ινσουλίνη","Διαβήτης","#Διαβήτης","διαβητικός","#διαβητικός","γλυκόζης στο αίμα","#γλυκόζηςστοαίμα","ζωή με διαβήτη",
     "#διαβήτηςτύπου1","#διαβήτης τύπου 2",
     "diabetiske","#diabetiske",
     "انسولین","#انسولین","ذیابیطس","#ذیابیطس","خون میں گلوکوز","#خون میں گلوکوز","ٹائپ 1 ذیابیطس","ٹائپ 2 ذیابیطس",
     "blodsocker","#blodsocker","#typ1diabetes","#typ2diabetes",
     "Bệnh tiểu đường","#Bệnhtiểuđường","mắc bệnh đái đường","#mắcbệnhđáiđường","đường huyết","#đường huyết","bệnh tiểu đường loại 1",
     "bệnh tiểu đường loại 2",
     "Diabet","#Diabet","glucoza din sange","#glucozadinsange","diabet de tip 1","diabet de tip 2",
     "insulini","#insulini","kisukari","#kisukari","sukari kwenye damu","kisukari type 1 ","kisukari type 2",
     "ciwon sukari","#ciwonsukari","jini glucose","#jiniglucose",
     "insulien","#insulien","suikersiekte","#suikersiekte","diabeet","#diabeet","bloedglukose","#bloodglucose",
     "ኢንሱሊን","#ኢንሱሊን","የስኳር","#የስኳር","የሱካር በሽታ","#የሱካር በሽታ","የግሉኮስ","#የግሉኮስ"
]
            
            
#Fill keys and tokens, 
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

class StreamListener(tweepy.StreamListener): #tweepy class to access the Twitter Streaming API    

    def on_connect(self):
        print("You are now connected to the streaming API.") #Connection to the streaming API
 
    def on_error(self, status_code): #if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data): #connection to mongoDB and storing the tweet
        try:
    
            datajson = json.loads(data)  # Decode the JSON from Twitter
            
            created_at = datajson['created_at'] #grab the 'created_at' data from the Tweet to use for display

            print("Tweet collected at " + str(created_at))  #print out a message 
            
            db.Diabetes.insert(datajson) #insert the data into mongoDB into a collection called Diabetes 
                                                   
        except Exception as e:
           print(e)


#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
time.sleep(2)
print("Tracking: " + str(KEYWORDS))
while True:
    try:
        streamer.filter(track=KEYWORDS, stall_warnings = True)
    except ProtocolError:
        continue